In [1]:
from transformers import GPT2ForQuestionAnswering, GPT2Tokenizer, AdamW
from datasets import load_dataset
from tqdm import tqdm
import torch

model = GPT2ForQuestionAnswering.from_pretrained('aman-mehra/gpt2-large-finetune-squad-ep-2.0-lr-2e-06-wd-0.01')
tokenizer = GPT2Tokenizer.from_pretrained('aman-mehra/gpt2-large-finetune-squad-ep-2.0-lr-2e-06-wd-0.01')
dataset=load_dataset('squad',split='train')

In [2]:
def predict_answer(question, context):
    inputs = tokenizer(question, context, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)

    answer_start_index = outputs.start_logits.argmax()
    answer_end_index = outputs.end_logits.argmax()

    predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
    
    return tokenizer.decode(predict_answer_tokens)

In [3]:
answer = dataset['answers']
answer=answer[30000:40000]
print(len(answer))
lista_answer=[e['text']for e in answer]
lista_predicted=[]
for i in tqdm(range(30000,40000)):
    lista_predicted.append(predict_answer(dataset['question'][i], dataset['context'][i]))


10000


100%|██████████| 10000/10000 [6:26:02<00:00,  2.32s/it]  


In [4]:
import evaluate
rouge=evaluate.load('rouge')
predictions=lista_predicted
references=lista_answer
results=rouge.compute(predictions=predictions,references=references)
print(results)


{'rouge1': 0.7901998995327322, 'rouge2': 0.493331788506298, 'rougeL': 0.7898030470793811, 'rougeLsum': 0.7896041172500119}
